In [ ]:
# NOTE: An assumption is made that the user has brightway and a number of other libraries installed.
# Also, development was done using Python 3.7, while there shouldn't be any actual broken code,
#  older versions of Python are not considered.

from pathlib import Path

import brightway2 as bw
from src import Builder

In [ ]:
sorted(bw.projects)

In [ ]:
# Select a superstructure project
proj_name = "superstructure"
bw.projects.set_current(proj_name)

In [ ]:
sorted(bw.databases)

In [ ]:
test_dbs = ["A", "B"]

In [ ]:
# Initialize the builder.
# - (Optional) Creates the superstructure database.
# - Compares the superstructure database against the others.
# - Adds both missing activities and exchanges to the superstructure.
bld = Builder.superstructure_from_databases(test_dbs, "C")

In [ ]:
# Take the superstructure and delta databases
# - Construct a DataFrame from all biosphere, technosphere and production exchanges.
# NOTE: This might only fill in the '*_database', '*_key' and scenario columns.
#  Using the `finalize_superstructure` method will fill in the missing information.
bld.build_superstructure()

In [ ]:
# Complete superstructure scenario DataFrame of all exchanges
bld.superstructure

In [ ]:
# Filter the DataFrame of all exchanges where there is no difference.
# NOTE: This is not always required.
bld.filter_superstructure()

In [ ]:
# Filtered superstructure scenario DataFrame
bld.superstructure

In [ ]:
# Validate the keys in the superstructure.
bld.validate_superstructure()

In [ ]:
# Fill out the 'from' and 'to' columns with the 'key' information.
bld.finalize_superstructure()

In [ ]:
# The final draft of the superstructure scenario flows.
# NOTE: The first scenario column is the superstructure database itself.
# NOTE2: If the superstructure was built from scratch, all values will be 0.
#  Take this into account when saving/using this flow scenario dataframe.
bld.superstructure

In [ ]:
### Now, export both the superstructure database and the flow scenario file.
db = bw.Database("C")
db.process()
local_dir = Path.cwd()

In [ ]:
# For the superstructure database, use brightway2's 'BW2Package' format.
out_path = bw.BW2Package.export_obj(db, bld.name, "output")
out_file = Path(out_path)  # Capture the returned output path

In [ ]:
# Move the created package to the current working directory.
local_file = local_dir / out_file.name
if not local_file.exists():
    out_file.replace(local_file)

In [ ]:
# Now, save the flow scenarios to an excel file.
excel_file = local_dir / "{}_flow_scenarios.xlsx".format(bld.name)
bld.superstructure.to_excel(excel_file, index=False)